In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


Summary of text

In [ ]:
import sqlite3
import openai
import os

from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

def prompt(*args) -> str:
    text = args[0]
    try:
        chat_completion = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": text},
            ],
            max_tokens=150,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def main():
    # Create SQLite connection
    connection = sqlite3.connect(":memory:")

    # Register the custom function
    connection.create_function("prompt", 1, prompt)

    cursor = connection.cursor()

    # Create table and insert data
    cursor.execute("CREATE TABLE texts (id INTEGER PRIMARY KEY, content TEXT)")
    cursor.execute(
        "INSERT INTO texts (content) VALUES (?)",
        ('SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine.',)
    )

    # Execute query with custom function
    result = cursor.execute(
        "SELECT prompt('Summarize my text: ' || content) FROM texts"
    ).fetchone()[0]

    print("Summary:", result)

    # Close the connection
    cursor.close()
    connection.close()

if __name__ == "__main__":
    main()

Summary: SQLite is a compact, efficient, and reliable SQL database engine implemented as a C-language library.


Example-2

In [ ]:
import sqlite3
import openai
import os
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

def analyze_sentiment(*args) -> str:
    text = args[0]
    try:
        chat_completion = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant skilled in sentiment analysis."},
                {"role": "user", "content": f"Analyze the sentiment of this review and provide a brief summary: {text}"},
            ],
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def suggest_improvements(*args) -> str:
    product_name = args[0]
    reviews = args[1]
    try:
        chat_completion = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a product analyst who provides concise improvement suggestions."},
                {"role": "user", "content": f"Based on these reviews for {product_name}, what are the key areas for improvement? Reviews: {reviews}"},
            ],
            max_tokens=150,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def main():
    # Create SQLite connection
    connection = sqlite3.connect(":memory:")

    # Register custom functions
    connection.create_function("analyze_sentiment", 1, analyze_sentiment)
    connection.create_function("suggest_improvements", 2, suggest_improvements)

    cursor = connection.cursor()

    # Create tables
    cursor.executescript('''
        CREATE TABLE products (
            product_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            category TEXT,
            price DECIMAL(10,2),
            description TEXT,
            avg_rating DECIMAL(3,2)
        );

        CREATE TABLE reviews (
            review_id INTEGER PRIMARY KEY,
            product_id INTEGER,
            user_name TEXT,
            rating INTEGER CHECK(rating >= 1 AND rating <= 5),
            review_text TEXT,
            review_date DATE,
            helpful_votes INTEGER DEFAULT 0,
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        );
    ''')

    # Insert sample data (using abbreviated version for example)
    cursor.executescript('''
        INSERT INTO products (name, category, price, description, avg_rating) VALUES
        ('Sony WH-1000XM4', 'Electronics', 349.99, 'Wireless Noise Cancelling Headphones', 4.7),
        ('Instant Pot Duo', 'Kitchen', 99.99, '7-in-1 Electric Pressure Cooker', 4.8);

        INSERT INTO reviews (product_id, user_name, rating, review_text, review_date, helpful_votes) VALUES
        (1, 'AudioPhile', 5, 'Best noise cancelling headphones! Great sound but touch controls can be finicky.', '2024-01-10', 45),
        (1, 'TechReviewer', 4, 'Good sound quality but battery life could be better. Touch controls need improvement.', '2024-01-12', 32),
        (2, 'ChefJohn', 5, 'Amazing cooker but learning curve is steep. Wish the manual was better.', '2024-01-05', 67),
        (2, 'HomeChef', 4, 'Great for quick meals but pot size is limiting. Interface could be more intuitive.', '2024-01-08', 23);
    ''')

    # Analyze individual reviews
    print("\n=== Individual Review Sentiment Analysis ===")
    results = cursor.execute('''
        SELECT p.name, r.review_text, analyze_sentiment(r.review_text)
        FROM reviews r
        JOIN products p ON r.product_id = p.product_id
        ORDER BY p.product_id, r.review_date
    ''').fetchall()

    for product, review, sentiment in results:
        print(f"\nProduct: {product}")
        print(f"Review: {review}")
        print(f"Sentiment Analysis: {sentiment}")

    # Analyze product improvements based on all reviews
    print("\n=== Product Improvement Suggestions ===")
    results = cursor.execute('''
        SELECT
            p.name,
            suggest_improvements(p.name, GROUP_CONCAT(r.review_text, '. '))
        FROM products p
        JOIN reviews r ON p.product_id = r.product_id
        GROUP BY p.product_id
    ''').fetchall()

    for product, suggestions in results:
        print(f"\nProduct: {product}")
        print(f"Suggested Improvements: {suggestions}")

    # Close the connection
    cursor.close()
    connection.close()

if __name__ == "__main__":
    main()


=== Individual Review Sentiment Analysis ===

Product: Sony WH-1000XM4
Review: Best noise cancelling headphones! Great sound but touch controls can be finicky.
Sentiment Analysis: The sentiment of the review is overall positive. The reviewer highlights the excellent noise-cancelling capabilities and great sound quality of the headphones. However, they mention a downside regarding the touch controls being finicky, which introduces a slight negative note to their experience.

**Summary:** The reviewer praises the headphones for their outstanding noise cancelling and sound quality, but notes that the touch controls can be a bit unreliable.

Product: Sony WH-1000XM4
Review: Good sound quality but battery life could be better. Touch controls need improvement.
Sentiment Analysis: The sentiment of the review is mixed. While the reviewer appreciates the good sound quality, they express dissatisfaction with the battery life and the touch controls, indicating that there are areas for improvemen

Structured Output

In [ ]:
from pydantic import Field, create_model
import json
from typing import List, Type
import sqlite3
import openai
import os

from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

pydantic_types = {
    "VARCHAR": str,
    "INTEGER": int,
    "VARCHAR[]": List[str],
}


def create_pydantic_model(model_name: str, struct: dict, struct_descr: dict) -> Type:
    fields = {}

    for field_name, field_type in struct.items():
        fields[field_name] = (
            pydantic_types[field_type],
            Field(..., description=struct_descr[field_name]),
        )

    return create_model(model_name, **fields)


def prompt(*args) -> str:
    text = args[0]
    response_model = None

    if len(args) == 2:
        struct_dict = json.loads(args[1])
        struct = struct_dict["struct"]
        struct_descr = struct_dict["struct_descr"]

        response_model = create_pydantic_model("ResponseModel", struct, struct_descr)

    kwargs = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": text},
        ],
    }

    if response_model:
        kwargs["response_format"] = response_model

    try:
        if response_model:
            chat_completion = openai.beta.chat.completions.parse(**kwargs)
            if chat_completion.choices[0].message.parsed:
                return str(chat_completion.choices[0].message.parsed.model_dump_json())
        else:
            chat_completion = openai.chat.completions.create(**kwargs)
            return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def main():
    # Create SQLite connection
    connection = sqlite3.connect(":memory:")

    # Register the custom function
    connection.create_function("prompt", 2, prompt)

    cursor = connection.cursor()

      # Create table
    cursor.execute("CREATE TABLE posts (by TEXT, text TEXT, timestamp TEXT, summary TEXT);")

    # Corrected INSERT statement
    cursor.execute('''INSERT INTO posts (by, text, timestamp, summary) VALUES
        ('yrgulation', '&gt; In a word, gardening. It&#x27', '2022-08-22 18:37:49', 'Gardening brings joy, Micro-greens, a sweet delight, Nature''s calm embrace.'),
        ('paulmd', 'I actually don&#x27;t know what', '2022-08-22 18:37:46', 'Insurance wins on average, Claims raise future premiums, Risk smooths, costs remain.'),
        ('taylodl', '59% of Americans are correct, but', '2022-08-22 18:37:49', 'Fifty-nine percent, Worried for the children, Profit clouds their thoughts.'),
        ('0x457', 'Wrong or not, the point is tha made', '2022-08-22 18:37:33', 'XHTML''s strict rules, Errors led to chaos, Browsers struggled hard.'),
        ('rchaud', 'This is a US lawsuit, filed by a non', '2022-10-09 14:56:49', 'Lawsuit in the States, Non-Indians in the fray, Executives clash.')
    ''')
    # Fixed query with proper argument handling
    struct_json = '''{
        "struct": {
            "topic": "VARCHAR",
            "sentiment": "INTEGER",
            "technologies": "VARCHAR[]"
        },
        "struct_descr": {
            "topic": "topic of the post, single word",
            "sentiment": "sentiment of the post on a scale from 1 (neg) to 5 (pos)",
            "technologies": "technologies mentioned in the post"
        }
    }'''

    result = cursor.execute("""
        SELECT text, timestamp,
        prompt('Extract this data from the text: ' || text, ?) as my_output
        FROM posts
    """, (struct_json,)).fetchall()

    for row in result:
        print(row)
    # Close the connection
    cursor.close()
    connection.close()

if __name__ == "__main__":
    main()

('&gt; In a word, gardening. It&#x27', '2022-08-22 18:37:49', '{"topic":"gardening","sentiment":4,"technologies":[]}')
('I actually don&#x27;t know what', '2022-08-22 18:37:46', '{"topic":"Uncertainty","sentiment":2,"technologies":[]}')
('59% of Americans are correct, but', '2022-08-22 18:37:49', '{"topic":"Statistics","sentiment":3,"technologies":[]}')
('Wrong or not, the point is tha made', '2022-08-22 18:37:33', '{"topic":"Point","sentiment":3,"technologies":[]}')
('This is a US lawsuit, filed by a non', '2022-10-09 14:56:49', '{"topic":"Lawsuit","sentiment":3,"technologies":["US Legal System"]}')
